### Thesis notebook 4.2. - R_Gonz

### Aplication of SMOTE - Keeping Outliers

### Non-temporal data representation

In [1]:
#import necessary modules/libraries
import numpy as np
import scipy
import pandas as pd
import datetime as dt
import warnings
import time
import plotly.express as px

#tqdm to monitor progress
from tqdm.notebook import tqdm, trange
tqdm.pandas(desc="Progress")

#time related features
from datetime import timedelta
from copy import copy, deepcopy

from lightgbm import LGBMModel,LGBMClassifier

from imblearn.over_sampling import RandomOverSampler, SMOTE

warnings.filterwarnings('ignore')
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer,PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, RepeatedStratifiedKFold, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import mutual_info_classif, RFE, RFECV, SelectFromModel
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import AgglomerativeClustering , KMeans, DBSCAN

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, RidgeCV, LassoCV, ElasticNetCV

from sklearn.datasets import make_classification, load_digits
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, recall_score, classification_report, average_precision_score, precision_recall_curve

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#global variables that may come in handy
#course threshold sets the % duration that will be considered (1 = 100%)
duration_threshold = [0.1, 0.25, 0.33, 0.5, 1]

#colors for vizualizations
nova_ims_colors = ['#BFD72F', '#5C666C']

#standard color for student aggregates
student_color = '#474838'

#standard color for course aggragates
course_color = '#1B3D2F'

#standard continuous colormap
standard_cmap = 'viridis_r'

#Function designed to deal with multiindex and flatten it
def flattenHierarchicalCol(col,sep = '_'):
    '''converts multiindex columns into single index columns while retaining the hierarchical components'''
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col
    
#number of replicas - number of repeats of stratified k fold - in this case 10
replicas = 30

In [ ]:
#imports dataframes
course_programs = pd.read_excel("../Data/Modeling Stage/R_gonz_Non_temporal_Datasets.xlsx", 
                                dtype = {
                                    'course' : object,
                                    'userid' : object},
                               sheet_name = None)

#drop unnamed 0 column
for i in course_programs:
    course_programs[i].drop('Unnamed: 0', axis = 1, inplace = True)
    
#save tables 
class_list = pd.read_csv('../Data/Modeling Stage/R_Gonz_updated_classlist.csv', 
                         dtype = {
                                   'course': object,
                                   },
                        parse_dates = ['Week before start', 'Start Date',
                                       'End Date', 'Date_threshold_10',
                                      'Date_threshold_25', 'Date_threshold_33', 'Date_threshold_50',
                                      'Date_threshold_100']).drop('Unnamed: 0', axis = 1)

In [ ]:
course_programs['Date_threshold_10'].describe()

##  Stage: Data understanding Stage

#### INITIAL EXPLORATION, CLEANING & FEATURE ENGINEERING

In [ ]:
#Step1 - Take a look at the datesets
for i in course_programs:
    print(f'Time period: {i}\n\n{course_programs[i].head()}\n')

In [ ]:
#Step1 - Take a look at the datesets
for i in course_programs:
    print(f'Time period: {i}\n\n{course_programs[i].info()}\n')

In [ ]:
#describe method
for i in tqdm(course_programs):
    print(f'Time period: {i}\n\n{course_programs[i].describe(include = "all")}\n')

#### SOME DATA EXPLORATION

In [ ]:
#then we plot an histogram with all courses, we are not interested in keeping courses with a number of students inferior to 10
sns.set_theme(context='paper', style='whitegrid', font='Calibri', rc={"figure.figsize":(16, 10)}, font_scale=2)

#Plot the distributions of each feature 
for i in tqdm(course_programs):
    print(i)
    course_programs[i].hist(figsize=(16, 20), bins=100, xlabelsize=8, ylabelsize=8, color = student_color)

#### Converting target:

As stated, we will predict the exam grade as it does not depend direcly from assignment grades.

We will use a double classification problem:

**Problem 1**: Identify students at risk of failure. In the portuguese system, a student passes if the final grade greater or equal to 10. Students who do not meet the threshold fail. When it comes to exams, the difference is not as clear cut. For simplicity, we will consider exam grade to be less or equal to 10 - same with final grade. 

**Problem 2**: Identify high performing students. The classification of high performers is not unanymous. A simple solution would be consider as high performers all students whose grade is greater or equal to a certain threshold (say 17/20). There are issues with this approach however:

    - different teachers have different criteria for grading and their grading decisions. We can look at the top 20% of students in each course as identify them as the high performers. 

In [ ]:
#create loops for targets
for i in tqdm(course_programs):
    print(i)
    
    #deal with risk of failure
    course_programs[i]['final_fail'] = np.where(course_programs[i]['final_mark'] > 5, 0, 1) #at risk in final grade   
    
    #deal with gifted classification
    course_programs[i]['final_gifted'] = np.where(course_programs[i]['final_mark'] >= 8.5, 1, 0) #gifted students according to final grade

In [ ]:
course_programs['Date_threshold_100'].describe(include = 'all').T

#### Looking for Group (Course) Aggregates

We will look at the 100% duration logs in order to make most of our verification. After all, the 100% duration threshold will be the basis for comparison.

We will, for now, use the following immediate criteria for exclusion:
1. Median % of days with no interaction > 80%,
2. To have 0 mean for at-risk students or gifted students.

In [ ]:
aggregates = course_programs['Date_threshold_100'].groupby('course').agg({
                                                                'userid' :  [('Number of users','nunique')],
                                                                'Number of sessions' : ['min', 'mean', 'median', 'max'],
                                                                'Clicks per day' : ['min', 'mean', 'median', 'max'],
                                                                'Days with no interaction (%)' : ['min', 'mean', 'median', 'max'], 
                                                                'final_fail' : 'mean',
                                                                'final_gifted' : 'mean',                                                                
                                                                    })
#show all
with pd.option_context("display.max_rows", None):
    display(aggregates)
    
        #same for session features
aggregates.columns = aggregates.columns.map(flattenHierarchicalCol)
aggregates.reset_index(inplace = True)

In [ ]:
#only keep rows that fulfill the criteria - median user with less than 80% days without interaction 
aggregates = aggregates[~((aggregates['Days with no interaction (%)_median'] > 85) | (aggregates['final_fail_mean'] == 0) | (aggregates['final_gifted_mean'] == 0))]
courses_to_keep = aggregates['course'].unique()

#filtering dataset for all course durations
for i in tqdm(course_programs):
    course_programs[i] = course_programs[i][course_programs[i]['course'].isin(courses_to_keep)].reset_index(drop = True)

course_programs['Date_threshold_100'].describe(include = 'all').T

In [ ]:
course_programs['Date_threshold_100'].columns

In [ ]:
# Organizar features
targets = ['final_fail', 'final_gifted']

numeric_feat = [ 'Number of clicks', 'Number of sessions',
       'Largest period of inactivity (h)', 'Total time online (min)',
       'Average session duration (min)', 'Start of Session 1 (%)',
       'Start of Session 2 (%)', 'Start of Session 3 (%)',
       'Start of Session 4 (%)', 'Start of Session 5 (%)',
       'Start of Session 6 (%)', 'Start of Session 7 (%)',
       'Start of Session 8 (%)', 'Start of Session 9 (%)',
       'Start of Session 10 (%)', 'Days with no interaction',
       'Clicks on forum', 'Links viewed', 'Clicks on folder',
       'Clicks on course', 'Resources viewed', 'Discussions viewed',
       'Quizzes started', 'Files downloaded', 'Assignments submitted',
       'Assignments viewed', 'Forum posts', 'Number of days', 'Clicks per day',
       'Clicks per session', 'Clicks (% of course total)',
       'Submissions (% of course total)', 'Days with no interaction (%)',
        'On/off campus click ratio', 'Clicks on campus']

#optional features -> assignment grades
#optional_feats =[]

#binary_feat = []

#categorical_feat = []

#date = []

drop_feat = ['final_mark', 'Average grade of assignments',]

In [ ]:
#Data Quality Report for Continuous Features

def DescribeContinuousFeatures(Continuous_Features, dataset):
    Continuous_Head = ['Count', 'Missing Values (%)', 'Cardinality', 'Minimum', '1st Qrt.', 'Mean', 'Median', '3rd Qrt.', 'Maximum', 'Std. Dev.']
    Continuous_Describe = pd.DataFrame(index=Continuous_Features, columns=Continuous_Head)
    Continuous_Describe.index.name = 'Feature Name'
    columns = dataset[Continuous_Features]

#Total Number of Instances
    Continuous_Describe[Continuous_Head[0]] = columns.count()

#Percentage of instances that has Missing Values (sabendo à partida que nenhuma variável contínua tem missings)
    Continuous_Describe[Continuous_Head[1]] = columns.isnull().sum() * 100 / len(dataset)
    
#Cardinality of each feature (cardinality measures the number of Distinct Values)
    Continuous_Describe[Continuous_Head[2]] = columns.nunique()

#Minimum Value
    Continuous_Describe[Continuous_Head[3]] = columns.min()

#1ST Quartile
    Continuous_Describe[Continuous_Head[4]] = columns.quantile(0.25)

#Mean
    Continuous_Describe[Continuous_Head[5]] = round(columns.mean(), 2)

#Median
    Continuous_Describe[Continuous_Head[6]] = columns.median()

#3rd Quartile
    Continuous_Describe[Continuous_Head[7]] = columns.quantile(0.75)

#Maximum Value
    Continuous_Describe[Continuous_Head[8]] = columns.max()

#Standard Deviation
    Continuous_Describe[Continuous_Head[9]] = round(columns.std(),2)
    
    return Continuous_Describe

for i in tqdm(course_programs):
    print(f'{i}\n\n{DescribeContinuousFeatures(numeric_feat,course_programs[i])}\n')

The cell below may be commented for faster computation.

In [ ]:
# sns.set_theme(context='paper', style='whitegrid', font='Calibri', font_scale=2)

# for i in tqdm(course_programs):
    
#     print(i)
#     #perform a very simple pairplot
#     g = sns.PairGrid(course_programs[i][numeric_feat], corner = True)
    
#     #frequency histogram on diagonal
#     g.map_diag(sns.histplot, color = 'grey', stat = 'frequency', kde = True)
    
#     #regplot is a scatter with regression line
#     g.map_offdiag(sns.regplot, fit_reg=True, x_jitter=.1, color = course_color,  marker = 'x')
    
#     # Layout
#     plt.subplots_adjust(top=0.95)
#     plt.title("Pairwise Relationship of Numerical Variables", fontweight="bold")
    
#     plt.savefig(f'../Images/R_Gonz_numerical_feats_{i}_pair.png', transparent=True, dpi=300)
#     plt.close("all")

In [ ]:
# def cor_heatmap(cor):
#     plt.figure(figsize=(32,32))
#     return sns.heatmap(data = cor, annot = True, cmap = standard_cmap, fmt='.1',
#                 vmin = -1, vmax = 1,
#                )
    
# sns.set_theme(context='paper', style='whitegrid', font='Calibri', rc={"figure.figsize":(32, 32)}, font_scale=2)

# for i in tqdm(course_programs):
#     print(i)
#     cor_spearman = course_programs[i][numeric_feat].corr(method = 'spearman')
#     g = cor_heatmap(cor_spearman)
#     fig = g.get_figure()
#     fig.savefig(f'../Images/R_gonz_numerical_feats_{i}_heat.png', transparent=True, dpi=300)
#     plt.close("all")
    
# del g, fig

### Immediate observations

Very low correlations throughout most metric features, eith the exception of a couple.
Still need to verify what to do concerning outliers.

## Functions to Call later

In [ ]:
def feature_selection_table(dataset, target):

    feature_df = dataset.copy()
    
    X = feature_df.drop(targets,axis=1)
    y = feature_df[target]
    num_feats=len(X.columns)
    X_norm = StandardScaler().fit_transform(X)

    #RFE
    rfe_selector = RFE(estimator=DecisionTreeClassifier(), step=2)
    rfe_selector.fit(X_norm, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:,rfe_support].columns.tolist()

    #RFECV
    rfecv_selector = RFECV(estimator=DecisionTreeClassifier(), step=1, cv=5, scoring='f1_weighted',
                          min_features_to_select = 10)
    rfecv_selector = rfecv_selector.fit(X_norm, y)
    rfecv_support = rfecv_selector.support_
    rfecv_feature = X.loc[:,rfecv_support].columns.tolist()

    #Logistic regression
    embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
    embeded_lr_selector.fit(X_norm, y)
    embeded_lr_support = embeded_lr_selector.get_support()
    embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()

    #RandomForest 
    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
    embeded_rf_selector.fit(X, y)
    embeded_rf_support = embeded_rf_selector.get_support()
    embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()


    #LGBMClassifier
    lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
                reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
    embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
    embeded_lgb_selector.fit(X, y)
    embeded_lgb_support = embeded_lgb_selector.get_support()
    embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()

    #Lasso
    reg = LassoCV(random_state=123) 
    reg.fit(X=X,y=y)
    lasso_coef = pd.Series(reg.coef_,index = X.columns)

    #Ridge
    ridge = RidgeCV()
    ridge.fit(X=X,y=y)
    coef_ridge = pd.Series(ridge.coef_,index = X.columns)

    #Elastic 
    elast = ElasticNetCV(cv=5, random_state=123)
    elast.fit(X, y)
    elast_coef = pd.Series(elast.coef_,index = X.columns)

    # put all selection together
    feature_selection_df = pd.DataFrame({'Feature':X.columns, 'RFE':rfe_support,'RFECV':rfecv_support, 'Logistics':embeded_lr_support,
                                        'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support, 'Lasso':lasso_coef != 0, 'Ridge':coef_ridge != 0, 'Elastic':elast_coef != 0})
    # count the selected times for each feature
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    
    # display the top 100
    feature_selection_df = feature_selection_df.sort_values(['Total'] , ascending=False)
    feature_selection_df.set_index('Feature',inplace=True)
    feature_selection_df
    
    return feature_df,feature_selection_df

In [ ]:
def normalize(train, test, scaler):
    
    if scaler == 'MinMax':
        pt = MinMaxScaler()
    elif scaler == 'Standard':
        pt = StandardScaler()
    elif scaler == 'Robust':
        pt = RobustScaler()
    elif scaler == 'Quantile':
        pt = QuantileTransformer()
    else:
        pt = PowerTransformer(method='yeo-johnson')
    
    data_train = pt.fit_transform(train)
    data_test = pt.transform(test)
    # convert the array back to a dataframe
    normalized_train = pd.DataFrame(data_train,columns=train.columns)
    normalized_test = pd.DataFrame(data_test,columns=test.columns)
        
    return normalized_train, normalized_test

# MODEL TESTING & SELECTION

The first part of this section will hinge on defining the functions to use during model selection.

In [ ]:
#run desired model
def run_model(model_name, X, y):
    
    ###Baseline Classifier - most frequent class
    if model_name == 'Baseline - Majority Class':
        model = DummyClassifier(strategy="most_frequent").fit(X, y)
    if model_name == 'KNN':
        #weights to attribute higher weight to closer neighbors; seems to improve score highly
        model = KNeighborsClassifier(n_neighbors=10, weights='distance').fit(X, y) #
    if model_name == 'LR':
        model = LogisticRegression(tol=1e-05, solver='liblinear', penalty='l1', max_iter =200).fit(X, y) #
    if model_name == 'NB': 
        model = GaussianNB().fit(X, y)
    if model_name == 'BNB': #
        model = BernoulliNB().fit(X, y)
#     if model_name == 'MNB': #
#         model = MultinomialNB().fit(X, y)
    if model_name == 'MLP': #
        model = MLPClassifier(alpha=0.01, hidden_layer_sizes = (20,20), activation = 'relu', solver = 'adam', learning_rate = 'adaptive', verbose = 0, learning_rate_init = 0.02).fit(X, y)
    if model_name == 'CART DT':
        model = DecisionTreeClassifier(criterion='gini', max_depth=10).fit(X, y)
    if model_name == 'J48 DT':
        model = DecisionTreeClassifier(criterion = "entropy", max_depth=10).fit(X, y)        
    if model_name == 'SVM': #
        model = svm.SVC(tol = 0.01, probability = True, gamma='scale', kernel='rbf', C = 1).fit(X, y)
    
    ###ENSEMBLES
    if model_name == 'RF':
        model = RandomForestClassifier(max_depth = 10, random_state = 15, n_estimators=500, min_samples_leaf = 3).fit(X, y) #max_features=6, #max_depth é super imp para reduzir overfitting! #min_samples_lead highly reduces overfitting!
    if model_name == 'AdaBoost':
        model = AdaBoostClassifier(n_estimators = 95, learning_rate = 0.8, random_state = 15).fit(X, y) #importante que GSearchCV em DT maximizado!
    if model_name == 'GBoost':
        model = GradientBoostingClassifier(n_estimators=175, learning_rate=0.1, random_state=15).fit(X, y)
    if model_name == 'ExtraTree':
        model = ExtraTreesClassifier(n_estimators=175, criterion='entropy', max_depth = 10, min_samples_split= 50).fit(X, y)
    return model

In [ ]:
#averages scores of each run (for the present model) in each iteration of Repeated 10-fold CV that has been called
def avg_score(method,X,y, model_name):
    
    f1micro_train = []
    f1micro_val = []
    precision_train = []
    precision_val = []
    recall_train = []
    recall_val = []
    timer = []
    cm_holder = []
    test_holder = []
    auc_train = []
    auc_val = []
    features = []
    averaged_confusion_matrix=None
    
    for train_index, val_index in method.split(X,y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        #perform feature selection
        feature_df_dict,feature_selection_df_dict = feature_selection_table(data_prep_df.loc[X_train.index], k)
        
        #adjust train_val_total features and X_test features to the results of global feature selection
        X_train = feature_df_dict[feature_selection_df_dict.loc[feature_selection_df_dict['Total'] >3].index.tolist()]
        X_val = X_val.reindex(columns= X_train.columns)
        
        #append
        features.append(X_train.columns)
        
        #scale with standard scaler
        X_train, X_val = normalize(X_train, X_val,'Standard')
        
        #Oversampling with SMOTE
        over = SMOTE()
        X_train, y_train = over.fit_resample(X_train, y_train)
        
        begin = time.perf_counter()
        model = run_model(model_name, X_train, y_train)
        end = time.perf_counter()
        
        labels_train = model.predict(X_train)
        labels_val = model.predict(X_val)
        
        f1micro_train.append(f1_score(y_train, labels_train, average='micro'))
        f1micro_val.append(f1_score(y_val, labels_val, average='micro'))
        
        precision_train.append(precision_score(y_train, labels_train))
        precision_val.append(precision_score(y_val, labels_val))
        
        recall_train.append(recall_score(y_train, labels_train))
        recall_val.append(recall_score(y_val, labels_val))
        
        auc_train.append(roc_auc_score(y_train,model.predict_proba(X_train)[:,1]))
        auc_val.append(roc_auc_score(y_val,model.predict_proba(X_val)[:,1]))
        
        timer.append(end-begin)
        
        cm_holder.append(confusion_matrix(y_val, labels_val))
    
    #scale with standard scaler
    model = run_model(model_name, X_scaled,y_sampled)
    labels_test = model.predict(X_test)
    
    f1micro_test = f1_score(y_test, labels_test, average='micro')
    precision_test = precision_score(y_test, labels_test)
    recall_test = recall_score(y_test, labels_test)
    
    #SVM does not allow probabilities
    try:
        auc_test = roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
        
    except:
        auc_test = np.nan
    
    print(f'Classification Report for {model_name}:\nTest Data\n{classification_report(y_test, labels_test)}\n' + 
         f'Confusion Matrix:\n {confusion_matrix(y_test, labels_test)}\n')
    
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_f1_train = round(np.mean(f1micro_train),3)
    avg_f1_val = round(np.mean(f1micro_val),3)
    avg_f1_test = round(np.mean(f1micro_test),3)
    avg_precision_train = round(np.mean(precision_train),3)
    avg_precision_val = round(np.mean(precision_val),3)
    avg_precision_test = round(precision_test,3)
    avg_recall_train = round(np.mean(recall_train),3)
    avg_recall_val = round(np.mean(recall_val),3)
    avg_recall_test = round(recall_test,3)
    avg_auc_train = round(np.mean(auc_train),3)
    avg_auc_val = round(np.mean(auc_val),3)
    avg_auc_test = round(auc_test,3)
    
    
    std_time = round(np.std(timer),3)
    std_f1_train = round(np.std(f1micro_train),3)
    std_f1_val = round(np.std(f1micro_test),3)
    std_precision_train = round(np.std(precision_train),3)
    std_precision_val = round(np.std(precision_val),3)
    std_recall_train = round(np.std(recall_train),3)
    std_recall_val = round(np.std(recall_val),3)
    std_auc_train = round(np.std(auc_train),3)
    std_auc_val = round(np.std(auc_val),3)
    
    #from sklearn.metrics import cohen_kappa_score
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_f1_train) + '+/-' + str(std_f1_train), str(avg_f1_val) + '+/-' + str(std_f1_val), str(avg_f1_test), str(avg_precision_train) + '+/-' + str(std_precision_train), str(avg_precision_val) + '+/-' + str(std_precision_val), str(avg_precision_test), str(avg_recall_train) + '+/-' + str(std_recall_train), str(avg_recall_val) + '+/-' + str(std_recall_val), str(avg_recall_test), str(avg_auc_train)+ '+/-' + str(std_auc_train), str(avg_auc_val)+ '+/-' + str(std_auc_val), str(avg_auc_test), features, model_name

In [ ]:
def plt_bar(models, f1micro_train, f1micro_val, f1micro_test, path, date, target):
    
    sns.set_theme(context='paper', style='whitegrid', font='Calibri', font_scale=2)
    
    #Creates a figure and a set of subplots
    fig, ax = plt.subplots(figsize = (20, 12))

    #set width of bar
    barwidth = 0.3

    #set position of bar on X axis
    pos_train = np.arange(len(f1micro_test))
    pos_val = np.arange(len(f1micro_test))+0.3
    pos_test = np.arange(len(f1micro_test))+0.6
    
    #convert to number
    f1micro_train = [float(i.split('+')[0]) for i in f1micro_train]
    f1micro_val = [float(i.split('+')[0]) for i in f1micro_val]
    f1micro_test = [float(i.split('+')[0]) for i in f1micro_test]
    
    #makes the plot
    plt.bar(pos_train, f1micro_train, color= nova_ims_colors[0], width=barwidth, edgecolor='white', label='Train')
    plt.bar(pos_val, f1micro_val, color=course_color, width=barwidth, edgecolor='white', label='Validation')
    plt.bar(pos_test, f1micro_test, color=student_color, width=barwidth, edgecolor='white', label='Test')
    
    #sets x, y labels
    ax.set(xlabel = 'Model', ylabel = 'Accuracy')

    #sets x ticks locations and designation
    ax.set_xticks((pos_train+pos_val+pos_test)/3)
    ax.set_xticklabels(models, rotation='vertical')
    
    #ads title to the plot
    plt.title(f'10-fold Repeated Cross-Validation Results\nData: {date}, Target:{target}', fontweight="bold")
    plt.ylim([0.0, 1.05])
    #removes box to make the plot prettier
    plt.box(on=None)

    #Creates (pretty) legend
    plt.legend(frameon=False)
    
    plt.savefig(path, transparent=True, dpi=300)
    plt.close("all")

In [ ]:
def plot_roc_pr(models, X, y, path_roc, path_pr, date, target):  
    
    sns.set_theme(context='paper', style='whitegrid', font='Calibri', font_scale=2)
    # Below for loop iterates through your models list
    for m in models:
        model = m['model']
        y_pred=model.predict(X) # predict the test data
    #Compute False postive rate, and True positive rate
        fpr, tpr, _ = roc_curve(y, model.predict_proba(X)[:,1])
    #Calculate AUC
        auc = roc_auc_score(y,model.predict_proba(X)[:,1])
    #Plot
        plt.plot(fpr, tpr, label='%s ROC (area = %0.4f)' % (m['label'], auc))
    #Makes it pretty!
    #plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Specificity (False Positive Rate)', fontweight = 'bold')
    plt.ylabel('Sensitivity (True Positive Rate)', fontweight = 'bold')
    plt.title(f'ROC Curve Test\nData: {date}, Target:{target}', fontweight = 'bold')
    plt.legend(loc="lower right", frameon=False)
    #save fig
    plt.savefig(path_roc, transparent=True, dpi=300)
    plt.close("all")

    
    # Below for loop iterates through your models list
    for m in models:
        model = m['model']
        y_pred=model.predict(X) # predict the test data
    #Compute Precision and Recall
        precision, recall, _ = precision_recall_curve(y, model.predict_proba(X)[:,1])
    #Calculate AP
        ap = average_precision_score(y, model.predict_proba(X)[:,1])
    #Plot
        plt.plot(recall, precision, label='%s AP (area = %0.4f)' % (m['label'], ap))
    #Makes it pretty!
    plt.xlim([-0.05, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall (Positive Predictive Value)', fontweight="bold")
    plt.ylabel('Precision (True Positive Rate)', fontweight="bold")
    plt.title(f'Precision-Recall Curve Test\nData: {date}, Target:{target}', fontweight="bold")
    plt.legend(loc="lower left", frameon=False)
    
    #save fig
    plt.savefig(path_pr, transparent=True, dpi=300)
    plt.close("all")

### One more thing to do is to give a proper name to each plot label

In [ ]:
date_names = {
             'Date_threshold_10': '10% of Course Duration',   
             'Date_threshold_25': '25% of Course Duration', 
             'Date_threshold_33': '33% of Course Duration', 
             'Date_threshold_50': '50% of Course Duration', 
             'Date_threshold_100':'100% of Course Duration', 
            }

target_names = {
                'exam_fail' : 'At risk - Exam Grade',
                'final_fail' : 'At risk - Final Grade', 
                'exam_gifted' : 'High performer - Exam Grade', 
                'final_gifted': 'High performer - Final Grade'
                }

#### Now, we try and arrange everything in order to train the models we get for every combination of Date Thresholds and Targets.

In [ ]:
#creating main dicts
#feature selection 
feature_df_dict = {}
feature_selection_df_dict = {}
feature_columns = {}

#result storage
results_normal_models = {}

for i in tqdm(list(course_programs.keys())[1:]):
    print(i)
    
    #create subordinate dicts
    feature_df_dict[f'{(i)}'] = {}
    feature_selection_df_dict[f'{(i)}'] = {}
    results_normal_models[i] = {}
    feature_columns[i] = {}
    #prepare data for split
    data_prep_df = course_programs[i].drop(columns = drop_feat).set_index(['course', 'userid'])
    #for now, fill nans with 0
    data_prep_df = data_prep_df.fillna(0)
    
    #set x and y
    X = data_prep_df.drop(columns = targets)
    y = data_prep_df[targets]

    #first, we create dict where we will store results for every date_threshold and targe
    #now, we start working with each instance of target
    for k in tqdm(targets):
        print(k)
        
        #start with train test split
        X_train_val, X_test, y_train_val, y_test = train_test_split(X,
                                                  y[k],
                                                  test_size = 0.20,
                                                  random_state = 15,
                                                  shuffle=True,
                                                  stratify=y)
        
        #feature selection first
        feature_df_dict[i][k],feature_selection_df_dict[i][k] = feature_selection_table(data_prep_df.loc[X_train_val.index], k)
        
        #adjust train_val_total features and X_test features to the results of global feature selection
        X_train_val_total = feature_df_dict[i][k][feature_selection_df_dict[i][k].loc[feature_selection_df_dict[i][k]['Total'] >3].index.tolist()]
        X_test = X_test.reindex(columns= X_train_val_total.columns)
        
        #scaling to use onlyinside function - not on cv
        X_scaled, X_test = normalize(X_train_val_total, X_test, 'Standard')
        
        #Oversampling with SMOTE
        overs = SMOTE()
        X_scaled, y_sampled = overs.fit_resample(X_scaled, y_train_val)
        
        #runs each Repeated (10) 10-fold Cross-Validation in all tested models by calling function avg_score and prints resutls with multiple metrics for each
        method_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=replicas, random_state = 15)
        
        results_Dummy = avg_score(method_cv, X_train_val, y_train_val, 'Baseline - Majority Class')
        results_KNN = avg_score(method_cv, X_train_val, y_train_val, 'KNN')
        results_LR = avg_score(method_cv, X_train_val, y_train_val, 'LR')
        results_NB = avg_score(method_cv, X_train_val, y_train_val, 'NB')
        results_BNB = avg_score(method_cv, X_train_val, y_train_val, 'BNB')
        results_NN = avg_score(method_cv, X_train_val, y_train_val, 'MLP')
        results_CART_DT = avg_score(method_cv, X_train_val, y_train_val, 'CART DT')
        results_J48_DT = avg_score(method_cv, X_train_val, y_train_val, 'J48 DT')
        resultsSVM = avg_score(method_cv, X_train_val, y_train_val, 'SVM')
        results_RF = avg_score(method_cv, X_train_val, y_train_val, 'RF')
        results_AdaBoost = avg_score(method_cv, X_train_val, y_train_val, 'AdaBoost')
        results_GBoost = avg_score(method_cv, X_train_val, y_train_val, 'GBoost')
        results_Xtra = avg_score(method_cv, X_train_val, y_train_val, 'ExtraTree')

        results_models = [results_Dummy, results_KNN, results_LR, results_NB, results_BNB, results_NN, results_CART_DT, results_J48_DT, resultsSVM,
                            results_RF, results_AdaBoost, results_GBoost, results_Xtra]

        f1micro_train = []
        f1micro_val = []
        f1micro_test = []

        precision_train = []
        precision_val = []
        precision_test = []

        recall_train = []
        recall_val = []
        recall_test = []

        auc_train = []
        auc_val = []
        auc_test = []
        
        times = []
        feature_columns[i][k] = []
        
        #organizes data for futher plotting
        for j, model in enumerate(results_models):
            feature_columns[i][k].append(f'{results_models[j][14]}: {results_models[j][13]}',)
            f1micro_train.append(results_models[j][1])
            f1micro_val.append(results_models[j][2])
            f1micro_test.append(results_models[j][3])
    
            precision_train.append(results_models[j][4])
            precision_val.append(results_models[j][5])
            precision_test.append(results_models[j][6])
    
            recall_train.append(results_models[j][7])
            recall_val.append(results_models[j][8])
            recall_test.append(results_models[j][9])
            
            auc_train.append(results_models[j][10])
            auc_val.append(results_models[j][11])
            auc_test.append(results_models[j][12])
            
            times.append(results_models[j][0])
            
        result = [f1micro_train, f1micro_val, f1micro_test, precision_train, precision_val, precision_test, recall_train, recall_val, recall_test, auc_train, auc_val, auc_test, times]

        results_normal_models[i][k] = pd.DataFrame(result, index = ['Train Accuracy (F1 Score micro)', 'Validation Accuracy (F1 Score micro)', 'Test Accuracy (F1 Score micro)', 'Precision Train', 'Precision Validation', 'Precision Test', 'Recall Train', 'Recall Validation', 'Recall Test', 
                                                                    'AUC - Train', 'AUC - Validation', 'AUC - Test', 'Time'],  
                                                   columns = ['Baseline - Majority Class', 'KNN', 'LR', 'NB', 'BNB', 'NN', 'CART DT', 'J48 DT', 'SVM', 'RF', 'AdaBoost', 'GBoost', 'ExtraTree'])
            
       
        #setting up paths for plots
        path_bar = f'../Images/R_Gonz/Non temporal models/{i}/normal_no_assign_results_bar_{k}_SMOTE.png'
        path_pr = f'../Images/R_Gonz/Non temporal models/{i}/normal_no_assign_precision_recall_{k}_SMOTE.png'
        path_roc = f'../Images/R_Gonz/Non temporal models/{i}/normal_no_assign_roc_{k}_SMOTE.png'
        
        #get roc and precision recall curves
        # Add models to list of models to incorporte in ROC curve
        models = [{'label': 'Majority Class', 'model': run_model('Baseline - Majority Class', X_scaled, y_sampled),},
            {'label': 'KNN', 'model': run_model('KNN', X_scaled, y_sampled),},
          {'label': 'LR', 'model': run_model('LR', X_scaled, y_sampled),},
          {'label': 'NB','model': run_model('NB', X_scaled, y_sampled),},
          {'label': 'BNB', 'model': run_model('BNB', X_scaled, y_sampled),},
          {'label': 'MLP', 'model': run_model('MLP', X_scaled, y_sampled),},
          {'label': 'CART DT', 'model': run_model('CART DT', X_scaled, y_sampled),},
          {'label': 'J48 DT', 'model': run_model('J48 DT', X_scaled, y_sampled),},
          {'label': 'RF', 'model': run_model('RF', X_scaled, y_sampled),},
          {'label': 'AdaBoost', 'model': run_model('AdaBoost', X_scaled, y_sampled),},
          {'label': 'GBoost', 'model': run_model('GBoost', X_scaled, y_sampled),},
          {'label': 'ExtraTree', 'model': run_model('ExtraTree', X_scaled, y_sampled),}] 
            #SVM not included coz there's no probs!
    
        plot_roc_pr(models, X_test, y_test, path_roc, path_pr, date_names[i], target_names[k])
        
    # from pandas.io.parsers import ExcelWriter
    with pd.ExcelWriter(f"../Data/Modeling Stage/Results/R_Gonz/Non temporal models/simple_models_no_assign_results_{i}_{replicas}_replicas_SMOTE.xlsx") as writer:  
    #saving file for setor comercial
    
        for sheet in targets:
            results_normal_models[i][sheet].to_excel(writer, sheet_name=str(sheet))
            
    #saving feature selection    
    with pd.ExcelWriter(f"../Data/Modeling Stage/Feature Selection/R_Gonz_no_assign_feature_selection_{i}_no_test_SMOTE.xlsx") as writer:  
        #saving file for setor comercial
    
        for sheet in targets:
            feature_selection_df_dict[i][sheet].to_excel(writer, sheet_name=str(sheet))
        
    #saving feature selection    
    pd.DataFrame(feature_columns[i]).to_excel(f"../Data/Modeling Stage/Feature Selection/R_Gonz_no_assign_feature_selection_{i}_cross_val_SMOTE.xlsx")